<a href="https://colab.research.google.com/github/mayarachew/IronyDetectionInTweets/blob/main/irony_detection_pt_attention_bilstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#!cp -r "/content/drive/My Drive/sentiment_analysis/irony_detection_EN_taskA" "irony_detection_EN"
#!cp -r "/content/drive/My Drive/sentiment_analysis/irony_detection_pt2" "irony_detection_pt2"

In [3]:
#!pip install tensorflow_text
#!pip install attention

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout, Flatten, Embedding, SimpleRNN,LSTM,Bidirectional
from keras.layers.embeddings import Embedding
from attention import Attention
from keras.datasets import reuters
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from collections import Counter

In [5]:
path_train = "irony_detection_pt2/ironico-train.txt"
path_valid = "irony_detection_pt2/ironico-valid.txt"
path_test = "irony_detection_pt2/ironico-test.txt"

In [6]:
df_train = pd.read_csv(path_train, sep='\t',encoding="UTF-8",low_memory = False)
df_valid = pd.read_csv(path_valid, sep='\t',encoding="UTF-8",low_memory = False)
df_test = pd.read_csv(path_test, sep='\t',encoding="UTF-8",low_memory = False)

df_train.head()

,id,text,Label
0,7.728316e+17,"Jogo será em Manaus, muito mais longe que a Co...",ironico
1,8.658627e+17,Diferença de renda entre brancos enegros cresc...,nao-ironico
2,8.699588e+17,ECONOMIA CRISE RECESSÃO FORATEMER DIRETASJÁ: C...,nao-ironico
3,5.378169e+17,"Instagram, aquele programa de tirar foto da co...",ironico
4,7.330307e+17,• MARQUEM OS AMIGOS •bomdia boatarde boanoite ...,ironico


In [7]:
df_test['Label'] = pd.Categorical(df_test['Label'])
df_test['Label'] = df_test['Label'].cat.codes

df_train['Label'] = pd.Categorical(df_train['Label'])
df_train['Label'] = df_train['Label'].cat.codes

df_valid['Label'] = pd.Categorical(df_valid['Label'])
df_valid['Label'] = df_valid['Label'].cat.codes

In [8]:
y_train = df_train['Label']
X_train = df_train['text']

y_test = df_test['Label']
X_test = df_test['text']

y_valid = df_valid['Label']
X_valid = df_valid['text']

In [9]:
df_train.dtypes


id       float64
text      object
Label       int8
dtype: object

In [10]:
num_classes = 2

results = Counter()
df_train['text'].str.lower().str.split().apply(results.update)
df_valid['text'].str.lower().str.split().apply(results.update)
df_test['text'].str.lower().str.split().apply(results.update)
total_vocab_size = len(results)

In [12]:
soma = 0

for item in df_train['text']:
  soma += len(item)
media = soma/len(df_train['text'])
print(f'Em média, cada tweets possui {media} palavras.')

Em média, cada tweets possui 72.57977736549165 palavras.


Hiperparametros

In [13]:
vocab_size = 1000 # make the top list of words (common words)
embedding_dim = 40
max_length = 40
bs = 32
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # OOV = Out of Vocabulary

In [14]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

In [15]:
word_index = tokenizer.word_index
len(word_index)

21223

In [16]:
train_sequences = tokenizer.texts_to_sequences(X_train)

valid_sequences = tokenizer.texts_to_sequences(X_valid)

test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

X_valid_padded = pad_sequences(valid_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

X_test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


Verificando se está tudo correto

In [17]:
print(X_train.iloc[3])
print(X_train_padded[3])

Instagram, aquele programa de tirar foto da comida, sqn!  tirando onda zuacao … 
[782 350 473   2 426 294  11 933 147   1   1   1 175   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]


In [18]:
X_train_padded.shape, X_valid_padded.shape

((10780, 40), (2310, 40))

In [19]:
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)
#y_test = to_categorical(y_test)

Segue o modelo LSTM

In [20]:
model = Sequential()
model.add(Embedding(vocab_size, output_dim=max_length, input_length=max_length))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Attention(max_length))
model.add(Dense(2))
model.add(Activation('sigmoid'))
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 40)            40000     
                                                                 
 bidirectional (Bidirectiona  (None, 40, 128)          53760     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 40, 128)           0         
                                                                 
 attention (Attention)       (None, 40)                26624     
                                                                 
 dense (Dense)               (None, 2)                 82        
                                                                 
 activation (Activation)     (None, 2)                 0         
                                                        

In [21]:
from tensorflow.keras.optimizers import SGD,Adam
#sgd = SGD(learning_rate=0.05)
#model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

opt = Adam(learning_rate=0.0001, decay=1e-6)
model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

#model.compile(optimizer='adam',
#                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#                   metrics=['accuracy'])

In [22]:
model.fit(X_train_padded,y_train,validation_data=(X_valid_padded,y_valid),epochs=10,batch_size=bs)

Epoch 1/10
337/337 [==============================] - 26s 50ms/step - loss: 0.4847 - accuracy: 0.8238 - val_loss: 0.3036 - val_accuracy: 0.8251
Epoch 2/10
337/337 [==============================] - 16s 47ms/step - loss: 0.1187 - accuracy: 0.9614 - val_loss: 0.0803 - val_accuracy: 0.9766
Epoch 3/10
337/337 [==============================] - 16s 47ms/step - loss: 0.0440 - accuracy: 0.9892 - val_loss: 0.0562 - val_accuracy: 0.9840
Epoch 4/10
337/337 [==============================] - 16s 47ms/step - loss: 0.0299 - accuracy: 0.9929 - val_loss: 0.0463 - val_accuracy: 0.9861
Epoch 5/10
337/337 [==============================] - 16s 47ms/step - loss: 0.0232 - accuracy: 0.9946 - val_loss: 0.0430 - val_accuracy: 0.9874
Epoch 6/10
337/337 [==============================] - 17s 49ms/step - loss: 0.0193 - accuracy: 0.9960 - val_loss: 0.0438 - val_accuracy: 0.9883
Epoch 7/10
337/337 [==============================] - 17s 50ms/step - loss: 0.0163 - accuracy: 0.9965 - val_loss: 0.0407 - val_accuracy:

In [23]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])
    plt.figure(figsize=(16, 8))
    plt.subplot(1, 2, 1)
    plot_graphs(history, 'accuracy')
    plt.ylim(None, 1)
    #plt.xticks(range(0,20))
    plt.subplot(1, 2, 2)
    plot_graphs(history, 'loss')
    plt.ylim(0, None)
    #plt.xticks(range(0,20))

In [24]:
# import classification_report
from sklearn.metrics import classification_report

_, test_acc = model.evaluate(X_test_padded, to_categorical(y_test), verbose=0)
print(test_acc)

y_pred = model.predict(X_test_padded, batch_size=bs, verbose=2)

# get the class with highest probability for each sample
y_pred = np.argmax(y_pred, axis=-1)

# get the classification report
print(classification_report(y_test, y_pred))

0.9909090995788574
73/73 - 2s - 2s/epoch - 23ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1902
           1       0.98      0.96      0.97       408

    accuracy                           0.99      2310
   macro avg       0.99      0.98      0.98      2310
weighted avg       0.99      0.99      0.99      2310

